# 根据要求导出`WRF-CMAQ`模拟数据
两大剖面的数据。用matlab存成三维数组，三个维度分别是水平位置、垂直高度和小时时刻，用这样的三维数组去装载各指标，包括**O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献**，例如O3浓度的三维数组是CO3（2点位,300米,4月2日13时）=75μg/m3。其中水平位置可以用1~n代表n个经纬度组合，再单独给一个对照表即可，例如2点位=（E 113.6°、N 29.1°）。

---
*@author: Evan*\
*@date: 2023-05-15*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../src/')
from findpoint import findpoint
import ModelEvaLib as me

In [2]:
lon1, lat1 = 116.6, 31.1
lon2, lat2 = 117.4, 30.1

lon3, lat3 = 116.1, 29.74
lon4, lat4 = 117.46, 30.85

lon5, lat5 = 116.8, 30.54
lon6, lat6 = 117.3, 30.5

In [3]:
chemFile = xr.open_dataset('F:/Data/case_anqing/April/April_chem.nc')
metFile  = xr.open_dataset('F:/Data/case_anqing/April/April_met.nc')
isam = xr.open_dataset('F:/Data/case_anqing/April/April_ISAM1.nc')
pa=xr.open_dataset('F:/Data/case_anqing/April/April_PA1.nc')

nlevel = 21
number = 50

lat = chemFile.latitude
lon = chemFile.longitude

O3   = chemFile.O3[:,:nlevel,:,:]
NO2  = chemFile.NO2[:,:nlevel,:,:]
PM25 = chemFile.PM25[:,:nlevel,:,:]
ht   = chemFile.HT[:,:nlevel,:,:]

# mda8   = O3.rolling(time=8).mean().resample({'time':'D'}).max(dim='time')
# NO2avg = NO2.resample({'time':'D'}).mean(dim='time')
# PMavg  = PM25.resample({'time':'D'}).mean(dim='time')
# htavg  = ht.resample({'time':'D'}).mean(dim='time')
# print(mda8.shape)

HADV = pa.HADV_O3[:,:nlevel,:,:]
ZADV = pa.ZADV_O3[:,:nlevel,:,:]
HDIF = pa.HDIF_O3[:,:nlevel,:,:]
VDIF = pa.VDIF_O3[:,:nlevel,:,:]
DDEP = pa.DDEP_O3[:,:nlevel,:,:]
CHEM = pa.CHEM_O3[:,:nlevel,:,:]

AQ  = isam.O3_AQ[:,:nlevel,:,:]
HF  = isam.O3_HF[:,:nlevel,:,:]
TL  = isam.O3_TL[:,:nlevel,:,:]
CZ  = isam.O3_CZ[:,:nlevel,:,:]
JJ  = isam.O3_JJ[:,:nlevel,:,:]
OTH = isam.O3_OTH[:,:nlevel,:,:]
BCO = isam.O3_BCO[:,:nlevel,:,:]
ICO = isam.O3_ICO[:,:nlevel,:,:]

## Line1

In [4]:
# 生成沿两个点连线的坐标
lons1 = np.linspace(lon1,lon2,num=number)
lats1 = np.linspace(lat1,lat2,num=number)

# 选取这些点在数据集中的最接近的网格点的数据
x_index = []
y_index = []

for i in range(number): 
    out_x, out_y = findpoint(lons1[i],lats1[i],O3)
    x_index.append(out_x)
    y_index.append(out_y)
O3_sel   = O3.isel(x=x_index, y=y_index)
NO2_sel  = NO2.isel(x=x_index, y=y_index)
PM25_sel = PM25.isel(x=x_index, y=y_index)
htsel    = ht.isel(x=x_index, y=y_index)

diag_O3   = np.diagonal(O3_sel, axis1=2, axis2=3)
diag_NO2  = np.diagonal(NO2_sel, axis1=2, axis2=3)
diag_PM25 = np.diagonal(PM25_sel, axis1=2, axis2=3)
diag_ht   = np.diagonal(htsel, axis1=2, axis2=3)

diag_O3.shape

(720, 21, 50)

In [5]:
HADV_sel  = HADV.isel(x=x_index, y=y_index)
ZADV_sel  = ZADV.isel(x=x_index, y=y_index)
HDIF_sel  = HDIF.isel(x=x_index, y=y_index)
VDIF_sel  = VDIF.isel(x=x_index, y=y_index)
DDEP_sel  = DDEP.isel(x=x_index, y=y_index)
CHEM_sel  = CHEM.isel(x=x_index, y=y_index)

diag_HADV  = np.diagonal(HADV_sel, axis1=2, axis2=3)
diag_ZADV  = np.diagonal(ZADV_sel, axis1=2, axis2=3)
diag_HDIF  = np.diagonal(HDIF_sel, axis1=2, axis2=3)
diag_VDIF  = np.diagonal(VDIF_sel, axis1=2, axis2=3)
diag_DDEP  = np.diagonal(DDEP_sel, axis1=2, axis2=3)
diag_CHEM  = np.diagonal(CHEM_sel, axis1=2, axis2=3)

diag_HADV.shape

(720, 21, 50)

In [6]:
AQ_sel  = AQ.isel(x=x_index, y=y_index)
HF_sel  = HF.isel(x=x_index, y=y_index)
TL_sel  = TL.isel(x=x_index, y=y_index)
CZ_sel  = CZ.isel(x=x_index, y=y_index)
JJ_sel  = JJ.isel(x=x_index, y=y_index)
OTH_sel = OTH.isel(x=x_index, y=y_index)
BCO_sel = BCO.isel(x=x_index, y=y_index)
ICO_sel = ICO.isel(x=x_index, y=y_index)

diag_AQ  = np.diagonal(AQ_sel, axis1=2, axis2=3)
diag_HF  = np.diagonal(HF_sel, axis1=2, axis2=3)
diag_TL  = np.diagonal(TL_sel, axis1=2, axis2=3)
diag_CZ  = np.diagonal(CZ_sel, axis1=2, axis2=3)
diag_JJ  = np.diagonal(JJ_sel, axis1=2, axis2=3)
diag_OTH = np.diagonal(OTH_sel, axis1=2, axis2=3)
diag_BCO = np.diagonal(BCO_sel, axis1=2, axis2=3)
diag_ICO = np.diagonal(ICO_sel, axis1=2, axis2=3)

diag_AQ.shape

(720, 21, 50)

In [7]:
df_line1 = pd.DataFrame(
    index=np.arange(1,51),
    data={'经度':lons1,'纬度':lats1}
)
df_line1.to_excel('D:/Download/newsec1.xlsx')

In [8]:
savemat(
    'D:/Download/newsec1.mat',
    {
    'height':diag_ht,
    'O3':diag_O3,
    'NO2':diag_NO2,
    'PM25':diag_PM25,
    'HADV':diag_HADV,
    'ZADV':diag_ZADV,
    'HDIF':diag_HDIF,
    'VDIF':diag_VDIF,
    'DDEP':diag_DDEP,
    'CHEM':diag_CHEM,
    'AQ':diag_AQ,
    'HF':diag_HF,
    'TL':diag_TL,
    'CZ':diag_CZ,
    'JJ':diag_JJ,
    'OTH':diag_OTH,
    'BCO':diag_BCO,
    'ICO':diag_ICO
    }
    )